In [2]:
using Random, Distributions, Statistics, Plots
gr() 

Plots.GRBackend()

In [37]:
#arqモデル
function extract_y_t(y_t_array,k)
    if k <= length(y_t_array)
        return y_t_array[(length(y_t_array)+1-k)]
    else
        return 0
    end
end

function ar_q(y_0, c, sigma, theta_array, y_length)
    y_t_array = []
    noise_array = rand(Normal(0,sigma) , y_length)
    for i in 1:y_length
        if i == 0
            append!(y_t_array,y_0)
        else
            y_t = 0
            for j in 1:length(theta_array)
                y_t += theta_array[j]*extract_y_t(y_t_array,j)
            end
            append!(y_t_array,c+y_t+noise_array[i])
        end
    end
    return y_t_array
end

function predict_ar_p(y_t_array,c, sigma, theta_array,h)
    for i in 1:h
        y_t = 0
        for j in 1:length(theta_array)
            y_t += theta_array[j]*extract_y_t(y_t_array,j)
        end
        append!(y_t_array,c+y_t)
    end
    return y_t_array
end

function predict_ar_p_interval(y_t_array,c, sigma, theta_array,h,N)
    y_h = y_t_array[(h+1):length(y_t_array)][h]
    y_pred_h = []
    for n in 1:N
        ar_h = ar_q(0, c, sigma, theta_array, h)
        append!(y_pred_h,ar_h[h])
    end
    mse = sum( [(y_pred_h[n]-y_h)^2 for n in 1:N])/N
    upper= y_h +1.96*sqrt(mse)
    lower= y_h - 1.96*sqrt(mse)
    return upper, lower
end

#k次の自己共分散の値を計算
function autocovariance(k, ma1_array)
    auto_cov = Statistics.cov(ma1_array[1:(size(ma1_array)[1])-k],ma1_array[1+k:(size(ma1_array)[1])],corrected = false)
    return auto_cov
end

autocovariance (generic function with 1 method)

In [40]:
ar_3 = ar_q(0, 3, 4, [0.5,0.3,-0.2], 20)
ar_3_pred =  predict_ar_p(ar_3, 3, 4, [0.5,0.3,-0.2], 20)
predict_ar_p_interval(ar_3, 3, 4, [0.5,0.3,-0.2], 20,100)

(16.202325903050284, -1.2021054471152608)